# CS 123A Project
## Building a protein secondary structure predictor
Rachel Stanik, Phuong Tran


## First Read in the data

In [36]:
file = open("dataset.txt", "r")

## Parse the data

Data must be in following format: <br>
'> Protein sequence name' <br>
'\n' <br>
'< amino acid sequence>' <br>
'\n' <br>
'< secondary structure>' <br>
'\n' <br>

In [37]:
def parse_data(file, sequences, structures):
    lines = file.readlines()
    new_seq = False
    new_struct = False
    full_seq = ""
    full_struct = ""
    
    #Parse the text
    for line in lines:
        #new protein sequence and structure
        if ">" in line:
            new_seq = True
            new_struct = False
            full_seq = ""
            full_struct = ""
            #print(line) # for debugging
            
        #end of the amino acid sequence
        elif new_seq and line == "\n":
            new_seq = False
            new_struct = True
            sequences.append(full_seq)
            #print(full_seq) #for debugging
            
        elif new_seq:
            full_seq += line.strip()
            
        #end of secondary structure
        elif new_struct and line == "\n":
            structures.append(full_struct)
            #print(full_struct) #for debugging
            #print() #for debugging
            new_seq = False
            new_struct = False
            
        elif new_struct:
            full_struct += line.strip()
    
sequences = []
structures = []
parse_data(file, sequences, structures)
print(len(sequences))
print(len(structures))

24
24
